In [1]:
from pyspark.sql import SparkSession
import pyspark.pandas as ps

E:\Felpipe\Trabajo\Ciencias de datos en general\KaggleChallenges\venv\lib\site-packages\pyspark\pandas\__init__.py:43: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
PG_URL  = 'jdbc:postgresql://localhost:5432/graphs'
PG_USER = 'spark_ingest'
PG_PASS = 'GYleZAI2pTBKJYl9W1PL'
PG_SCHEMA = 'raw'
PG_TABLE =  'saml_d'
JDBC_JAR = r"C:\spark\spark-4.0.1-bin-hadoop3\jars\postgresql-42.7.4.jar"  

In [3]:
spark = (
    SparkSession.builder
    .appName("ieee-fraud-jupyter")
    .config("spark.jars", JDBC_JAR)
    .config("spark.driver.extraClassPath", JDBC_JAR)
    .config("spark.executor.extraClassPath", JDBC_JAR)
    .config("spark.sql.ansi.enabled", "false") #Para poder usar la API de pandas pues no soporta modo ansi
    .getOrCreate()
)

In [4]:
sdf = (
    spark.read.format("jdbc")
    .option("url", PG_URL)
    .option("dbtable", f"{PG_SCHEMA}.{PG_TABLE}")  
    .option("user", PG_USER)
    .option("password", PG_PASS)
    .option("driver", "org.postgresql.Driver")
    .option("partitionColumn", "id")
    .option("lowerBound", "1")
    .option("upperBound", "10000000")
    .option("numPartitions", "6")
    .option("fetchsize", "10000")
    .load()
)

In [5]:
sdf.show(5)
sdf.printSchema()

+-------------------+----------+--------------+----------------+--------+----------------+-----------------+--------------------+----------------------+------------+-------------+--------------------+---+
|               time|      date|sender_account|receiver_account|  amount|payment_currency|received_currency|sender_bank_location|receiver_bank_location|payment_type|is_laundering|     laundering_type| id|
+-------------------+----------+--------------+----------------+--------+----------------+-----------------+--------------------+----------------------+------------+-------------+--------------------+---+
|2025-10-13 09:16:59|2022-12-15|    6112386593|      6912017659|  290.56|       UK pounds|        UK pounds|                  UK|                    UK|         ACH|            0|Normal_Small_Fan_Out| 44|
|2025-10-13 13:49:38|2022-11-21|     351681005|      6659671585|41642.98|       UK pounds|        UK pounds|                  UK|                    UK|      Cheque|            0|N

In [6]:
df = sdf.pandas_api(index_col="id")

In [7]:
df.head()

,time,date,sender_account,receiver_account,amount,payment_currency,received_currency,sender_bank_location,receiver_bank_location,payment_type,is_laundering,laundering_type
id,,,,,,,,,,,,
10759,2025-10-13 15:54:12,2022-11-17,9855632427,7472681116,2154.21,UK pounds,UK pounds,UK,UK,Cheque,0,Normal_Small_Fan_Out
10760,2025-10-13 11:53:08,2022-12-15,5022191310,1315794902,10023.32,UK pounds,UK pounds,UK,UK,Credit card,0,Normal_Fan_In
10761,2025-10-13 19:18:16,2022-12-11,2484750288,3855004887,476.51,UK pounds,UK pounds,UK,UK,Cheque,0,Normal_Small_Fan_Out
10762,2025-10-13 16:49:05,2022-12-04,4497724073,8967784895,3800.76,UK pounds,UK pounds,UK,UK,Cheque,0,Normal_Small_Fan_Out
10763,2025-10-13 12:14:18,2022-11-30,1159912211,24474863,4754.51,UK pounds,UK pounds,UK,UK,Debit card,0,Normal_Fan_In
